# Tableaux

Un tableau est similaire à un `array` en OCaml : c'est un ensemble de cases mémoires contiguës qui permet d'accéder en O(1) à un élément en position quelconque. Comme en OCaml, la taille d'un tableau ainsi que le type des ses éléments est fixée à sa création :

In [1]:
int t[10];  // créé un tableau de 10 entiers

Cela a pour effet d'allouer $10$ entiers dans la pile, pour un total de $10\times 4$ octets :

In [2]:
sizeof(t)

40

On peut modifier et accéder à un élément par son indice (qui commence à 0) :

In [3]:
t[0] = 42;  // modification du 1er élément

42

In [4]:
t[0]  // valeur du 1er élément

42

Si on dépasse de la taille d'un tableau, on obtient une erreur :

In [5]:
t[10]  // le dernier indice d'un tableau de taille n est n - 1

input_line_16:2:2: warning: array index 10 is past the end of the array (which contains 10 elements) [-Warray-bounds]
 t[10]  // le dernier indice d'un tableau de taille n est n - 1
 ^ ~~
input_line_7:2:2: note: array 't' declared here
 int t[10];  // créé un tableau de 10 entiers
 ^


0

**Exercice** : Définir un tableau de taille 20 qui contient les entiers de 0 à 19.

## Initialisation

On peut initialiser un tableau de la façon suivante :

In [6]:
float t[3] = {2.718, 3.14, 1.618};

Dans ce cas, il n'est pas obligatoire de donner la taille du tableau (qui est automatiquement déduit de l'initialisation) :

In [7]:
float t[] = {2.718, 3.14, 1.618};

`float t[]` signifie qu'on définit un tableau sans donner sa taille. Sans initialisation, on obtient une erreur : 

In [8]:
float t[];

input_line_20:2:8: error: definition of variable with array type needs an explicit size or an initializer
 float t[];
       ^


Interpreter Error: 

## Pointeur et tableau

Lorsqu'on utilise un tableau comme une valeur (par exemple quand il apparaît à droite d'une affectation de variable), le tableau est automatiquement converti en pointeur vers le 1er élément du tableau. On parle de ***pointer decay*** :

In [9]:
int tab[10];
&tab[0]  // adresse du 1er élément de tab

@0x7ffe09e55778

In [10]:
int* p = tab;  // lorsqu'il est utilisé comme une valeur, tab est converti en pointeur
p              // qui est en fait la même adresse que &tab[0]

@0x7ffe09e55778

`int* p = tab;` est donc équivalent à `int* p = &tab[0]`, et mets dans `p` l'adresse du 1er élément de `tab`.  
Le nom `tab` ressemble beaucoup à un pointeur vers le premier élément du tableau. Ce n'est cependant pas exactement un pointeur, comme on le voit en utilisant `sizeof` :

In [11]:
printf("taille de tab (tableau) : %lu\n", sizeof(tab));
printf("taille de p (pointeur) : %lu", sizeof(p));

taille de tab (tableau) : 40
taille de p (pointeur) : 8

Hormis avec `sizeof` et `&`, un tableau se comporte exactement comme un pointeur vers le 1er élément.

## Passage en argument

Lorsque l'on passe un tableau en argument d'une fonction, il y a *pointer decay* et la fonction récupère un pointeur. Une fonction sur un tableau pourra donc prendre `int*` en argument :

In [12]:
int somme(int* tab, int n) { 
// renvoie la somme des éléments du tableau dont le 1er élément est à l'adresse tabtab
// n est la taille du tableau
    int s = 0;
    for(int i = 0; i < n; i++) {
        s += tab[i];
    }
    return s;
}

int t[] = {1, 2, 3, 4};
somme(t, 4)

10

Il est nécessaire de donner le `n` en argument à `somme` pour connaître la taille du tableau. En effet, `sizeof(tab)` dans `somme` donnerait la taille d'une adresse (qui est 8 octets, soit 64 bits) et non pas la taille du tableau.  
De plus, il est possible d'utiliser `int tab[]` au lieu de `int* tab` en argument de `somme`, mais le comportement est exactement le même :

In [13]:
int somme(int tab[], int n) { // fait exactement la même chose que la fonction précédente
    int s = 0;
    for(int i = 0; i < n; i++) {
        s += tab[i];
    }
    return s;
}

Un tableau est donc toujours passé sous forme de pointeur (il n'existe pas de passage par copie pour les tableaux). On peut s'en servir pour modifier un tableau.

**Exercice** : 
1. Écrire une fonction `swap` pour échanger deux éléments d'un tableau d'entiers.  
```c
void swap(int* tab, int i, int j);
```
2. En déduire une fonction `reverse` inversant l'ordre des éléments d'un tableau d'entiers.

## Allocation dynamique de tableau

Il est possible de créer un tableau dans le tas avec `malloc` :

In [14]:
int* p = (int*)malloc(10*sizeof(int)); // création d'un tableau de 10 entiers dans le tas
p[0] = 42; // modification du 1er élément

42

In [15]:
p[0] // le 1er élément a bien été modifié

42

On obtient, comme précédément, un pointeur vers le 1er élément du tableau ainsi créé. On peut se servir d'une allocation dynamique pour créer un tableau dont la taille dépend d'une variable :

In [16]:
int* range(int n) { // renvoie un tableau contenant 0, ..., n - 1
    int* t = (int*)malloc(n*sizeof(int));
    for(int i = 0; i < n; i++) {
        t[i] = i;
    }
    return t;
}

In [17]:
int* t = range(5);
for(int i = 0; i < 5; i++) {
    printf("t[%d] = %d\n", i, t[i]);
}

t[0] = 0
t[1] = 1
t[2] = 2
t[3] = 3
t[4] = 4


Sans allocation dynamique, il ne serait pas possible de définir un tableau dont la taille dépend d'une variable (sauf à utiliser des VLA pour *Variable Length Array* qui sont hors programme et dont l'usage est de toute façon déconseillé) :

In [18]:
int n = 5;
int t[n]; 

input_line_35:3:5: error: variable length array declaration not allowed at file scope
int t[n]; 
    ^ ~


Interpreter Error: 

**Exercice**
1. Écrire une fonction `copy` de prototype `double* copy(double* t)` renvoyant une copie du tableau en argument, alloué dynamiquement.  
2. Pourquoi est-on obligé de créer un nouveau tableau avec `malloc` ici?